In [1]:
class UnionFindSet: 
    def __init__(self, n):
        self._parents = [i for i in range(n + 1)]
        self._ranks = [1 for i in range(n + 1)]
        
    def find(self, u):
        while u != self._parents[u]:
            self.parents[u] = self._parents[self._parents[u]]
            u = self._parents[u]
        return u 
    
    def union(self, u, v):
        pu, pv = self.find(u), self.find(v)
        if pu == pv:
            return False         
        if self._ranks[pu] < self._ranks[pv]:
            self._parents[pu] = pv
        elif self._ranks[pu] > self._ranks[pv]:
            self._parents[pv] = pu
        else:
            self._parents[pv] = pu
            self._ranks[pu] += 1
        return True 

## 399. Evaluate Division

Equantions are given in the format A/B = k, where A and B are variables represented as strings, and k is a real number (floating point number). Given some queries, return the answers. If the answer doest not exist, return -1.0

**Example:**

Given a / b = 2.0, b / c = 3.0

queries are: a/ c = ?, b/a = ?, a/e = ?, a/a = ?, x/x = ?. 

return [6.0, 0.5, -1.0, 1.0, -1.0]

According to the example above:

equations = [ ["a", "b"], ["b", "c"]],

values = [2.0, 3.0]

queries  = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], 
["x", "x"]]

In [1]:
import collections 

### Approach 1. Graph + DFS 

In [14]:
def calcEquation(equations: list, values: list, queries: list) -> list:
    
    def divide(x,y,visited):
        if x == y:
            return 1.0
        for n in g[x]:
            if n in visited: 
                continue
            visited.add(n)
            d = divide(n,y, visited)
            if d > 0: return d *g[n][x]
        return -1.0
    g = collections.defaultdict(dict)
    for (x,y), v in zip(equations, values):
        g[x][y] = v
        g[y][x] = 1.0/v
        
    ans = [divide(x, y, set()) if x in g and y in g else -1 for 
          x, y in queries]
    
    return ans 
    

In [3]:
equations = [["a", "b"], ["b", "c"]]

values = [2.0, 3.0]

g = collections.defaultdict(dict)

for (x,y), v in zip(equations, values):
    print(x,y)
    print(v)
    g[x][y] = v
    g[y][x] = 1.0/v
    
print(g)

a b
2.0
b c
3.0
defaultdict(<class 'dict'>, {'a': {'b': 2.0}, 'b': {'a': 0.5, 'c': 3.0}, 'c': {'b': 0.3333333333333333}})


In [12]:
def divide(x,y, visited):
    print('visited is ')
    print(visited)
    if x == y: 
        return 1.0
    visited.add(x)
    print('g[x] is ')
    print(g[x])
    for n in g[x]:
        print('n is ')
        print(n)
        if n in visited:
            continue
        visited.add(n)
        d = divide(n, y, visited)
        print('d is ')
        print(d)
        if d > 0: 
            return d*g[x][n]
    return -1.0 


In [13]:
x = "a"

y = "c"

visited = set()

divide(x,y,visited)

visited is 
set()
g[x] is 
{'b': 2.0}
n is 
b
visited is 
{'b', 'a'}
g[x] is 
{'a': 0.5, 'c': 3.0}
n is 
a
n is 
c
visited is 
{'c', 'b', 'a'}
d is 
1.0
d is 
3.0


6.0

In [16]:
x = "a"

y = "e"

visited = set()

divide(x,y,visited)

visited is 
set()
g[x] is 
{'b': 2.0}
n is 
b
visited is 
{'b', 'a'}
g[x] is 
{'a': 0.5, 'c': 3.0}
n is 
a
n is 
c
visited is 
{'c', 'b', 'a'}
g[x] is 
{'b': 0.3333333333333333}
n is 
b
d is 
-1.0
d is 
-1.0


-1.0

In [18]:
x in g and y in g

False

Time Complexity: O(e + q*e)

Space Complexity: O(e)

In [2]:
equations = [["a", "b"], ["b", "c"]]

values = [2.0, 3.0]

In [4]:
import collections
g = collections.defaultdict(dict)
for (x,y), v in zip(equations, values):
    g[x][y] = v
    g[y][x] = 1.0/v

In [6]:
x = "b"

for z in g[x]:
    print(z)

a
c


In [13]:
def calcEquation(equations: list, values: list, queries: list) -> list:
    g = collections.defaultdict(dict)
    
    for (x,y), v in zip(equations, values):
        g[x][y] = v
        g[y][x] = 1.0/v
        
    def divide(x, y, visited):
        if x == y:
            return 1.0
        visited.add(x)
        for z in g[x]:
            if z in visited:
                continue 
            d = divide(z, y, visited)
            if d > 0: 
                return d*g[x][z]
        return -1.0
    
    ans = []
    
    for x, y in queries:
        if x in g and y in g:
            ans.append(divide(x,y,set()))
        else:
            ans.append(-1.0)
    return ans

In [14]:
queries = [["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], 
          ["x", "x"]]

calcEquation(equations, values, queries)

[6.0, 0.5, -1.0, 1.0, -1.0]

### Approach 2: Union Find 

In [15]:
def calcEquation2(equations, values, queries):
    def find(x):
        if x != U[x][0]:
            px, pv = find(U[x][0])
            U[x] = (px, U[x][1] * pv)
        return U[x]
    
    def divide(x, y):
        rx, vx = find(x)
        ry, vy = find(y)
        if rx != ry: 
            return -1.0
        return vx/vy
    
    U = {}
    for (x,y), v in zip(equations, values):
        if x not in U and y not in U:
            U[x] = (y, v)
            U[y] = (y, 1.0)
        elif x not in U:
            U[x] = (y, v)
        elif y not in U:
            U[y] = (x, 1.0/v)
        else:
            rv, vx = find(x)
            ry, vy = find(y)
            U[rx] = (ry, v/vx * vy)
            
    ans = [divide(x, y) if x in U and y in U else -1 for x, y in queries]
    return ans     

In [16]:
calcEquation2(equations, values, queries)

[6.0, 0.5, -1, 1.0, -1]

## 742. Network Delay Time

There are N network nodes, labelled 1 to N. 

Given times, a list of travel times as directed edges tims[i] = (u, v, w), where u is the source node, v is the target node, and w is the time it takes for a signal to travel from source to target. 

Now, we send a signal from a certain node K. How long will it take for all nodes to receive the signal? If it is impossible, return -1. 

In [23]:
times = [[2,1,1], [2,3,1], [3,4,1]]

U = {}

for x, y, t in times:
    if x not in U and y not in U:
        U[y] = (x, t)
        U[x] = (x, 0)
    elif y not in U:
        U[y] = (x, t)
    elif x not in U:
        U[x] = (x, 0)
    
    

In [24]:
print(U)

{1: (2, 1), 2: (2, 0), 3: (2, 1), 4: (3, 1)}


In [25]:
def find(x):
    if x != U[x][0]:
        rx, rv = find(U[x][0])
        U[x] = (rx, rv+U[x][1])
    return U[x]
        
    

In [26]:
find(4)

(2, 2)

In [27]:
find(3)

(2, 1)

In [28]:
find(1)

(2, 1)

In [29]:
find(2)

(2, 0)

In [32]:
print([find(i) for i in range(1,5)])

[(2, 1), (2, 0), (2, 1), (2, 2)]


In [ ]:
def networkDelayTime(times: list, N: int, K:int) -> int:
    
    def find(x):
        if x != U[x][0]:
            rx, rv = find(U[x][0])
            U[x] = (rx, rv+U[x][1])
        return U[x]
    
    U = {}

    for x, y, t in times:
        if x not in U and y not in U:
            U[y] = (x, t)
            U[x] = (x, 0)
        elif y not in U:
            U[y] = (x, t)
        elif x not in U:
            U[x] = (x, 0)
            
    [find()]

## 684. Redundant Connection 

In this problem, a tree is an undirected graph that is connected and has no cycles. 

The given input is a graph that started as a tree with N nodes (with distinct values 1, 2, ..., N), with one additional edge added. The added edge has two different vertices chosen from 1 to N, and was not an edge that already existed. 

The resulting graph is given as a 2D-array of edges. Each element of edges is a pair [u, v] with u < v, that represents an undirected edge connecting nodes u and v. 

Return an edge that can be removed so that the resulting graph is a tree of N nodes. If there are multiple answers, return the answer that occurs last in the given 2D-array. The answer edge [u,v] should be in the same format, with u < v. 

**Example 1:**

Input: [[1,2], [1,3], [2,3]]

Output: [2,3]

**Example 2:**

Input: [[1,2], [2,3], [3,4], [1,4], [1,5]]

Output: [1,4]



In [ ]:
def findRedundantConnection(edges: list) -> list:
    
    